In [ ]:
# ========================================================
# 🔹 CHyPS v2.5: Classificação Híbrida (Versão com Correção Rápida)
# ========================================================
# Autor: Abraão Gualberto Nazário
# Orientação e Revisão Final: Gemini
# Data: 21/06/2025
# Nota da Versão: Removido 'stratify=y' para garantir a execução com qualquer dataset.
# --------------------------------------------------------

# --- SETUP DO AMBIENTE ---

try:
    import google.colab
    IN_COLAB = True
    print("🔹 Ambiente: Google Colab detectado.")
    # Instala libs no Colab
    !pip install -q pandas openpyxl scikit-learn spacy openai matplotlib
except ImportError:
    IN_COLAB = False
    print("🔹 Ambiente local detectado.")

# Imports
import pandas as pd
import numpy as np
import spacy
import matplotlib.pyplot as plt
import json
import time
import os
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import ConfusionMatrixDisplay
from openai import OpenAI

# --- Setup spaCy ---
try:
    nlp = spacy.load('pt_core_news_lg')
    print("✅ Modelo spaCy 'pt_core_news_lg' carregado.")
except OSError:
    print("⚠️ Modelo 'pt_core_news_lg' não encontrado. Tentando 'pt_core_news_md'...")
    try:
        nlp = spacy.load('pt_core_news_md')
    except OSError:
        print("📥 Baixando modelo 'pt_core_news_md'...")
        spacy.cli.download('pt_core_news_md')
        nlp = spacy.load('pt_core_news_md')
    print("✅ Modelo spaCy 'pt_core_news_md' carregado.")

# --- Setup da API do LLM (Groq) ---
api_key = os.getenv('GROQ_API_KEY')
if not api_key:
    print("⚠️ Variável de ambiente 'GROQ_API_KEY' não encontrada.")
    try:
        from google.colab import userdata
        api_key = userdata.get('GROQ_API_KEY')
        print("✅ Chave da API da Groq carregada do Colab Secrets.")
    except Exception:
        api_key = input("Insira sua chave de API da Groq manualmente: ")

llm_client = OpenAI(api_key=api_key, base_url="https://api.groq.com/openai/v1")
print("✅ Cliente da API Groq configurado.")

# ========================================================
# 🔹 FUNÇÕES DO ALGORITMO CHyPS
# ========================================================

def preprocess_with_spacy(text, nlp_model):
    doc = nlp_model(str(text).lower())
    lemmas = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct and not token.is_space]
    return ' '.join(lemmas)

def llm_classify_real(text_to_classify, llm_client, model_name):
    system_prompt = """
    Você é um especialista em classificação. Sua tarefa é analisar o texto e classificá-lo em UMA das seguintes categorias: '15' (Terra/Agrário), '16' (Paz/Acordos) ou '4' (Educação).
    Sua resposta DEVE ser um objeto JSON com dois campos: "classe" (o número da categoria como string) e "confianca" (um número de 0.0 a 1.0 representando sua certeza).
    Exemplo de resposta válida: {"classe": "15", "confianca": 0.9}
    """
    user_prompt = f'Texto para classificar:\n---\n{text_to_classify}\n---\nSua resposta em formato JSON:'
    try:
        response = llm_client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0,
            response_format={"type": "json_object"}
        )
        json_response = json.loads(response.choices[0].message.content)
        return str(json_response.get('classe', 'NI')), float(json_response.get('confianca', 0.0))
    except Exception as e:
        print(f"❌ Erro na chamada da API LLM: {e}")
        return 'NI', 0.0

def chamar_llm_com_retry(text, tentativas=3, pausa=2):
    for tentativa in range(tentativas):
        try:
            classe, confianca = llm_classify_real(text, llm_client, "llama3-8b-8192")
            if classe != 'NI':
                return classe, confianca
        except Exception as e:
            print(f"Tentativa {tentativa+1}/{tentativas} falhou com erro: {e}")
        time.sleep(pausa)
    return 'NI', 0.0

def chyps_classify(texto_original, texto_limpo, rf_model, vectorizer, limiar_tau):
    texto_tfidf = vectorizer.transform([texto_limpo])
    rf_pred_proba = rf_model.predict_proba(texto_tfidf)
    confianca_rf = np.max(rf_pred_proba)
    predicao_rf = rf_model.classes_[np.argmax(rf_pred_proba)]
    predicao_llm, confianca_llm = chamar_llm_com_retry(texto_original)
    comprimento = len(texto_original.split())
    W_rf, W_llm = (0.3, 0.7) if comprimento > limiar_tau else (0.7, 0.3)
    score_rf = confianca_rf * W_rf
    score_llm = confianca_llm * W_llm
    return predicao_llm if score_llm > score_rf else predicao_rf

def plotar_resultados(resultados_df):
    plt.style.use('seaborn-v0_8-whitegrid')
    ax = resultados_df.plot(kind='bar', figsize=(14, 8), width=0.8, colormap='viridis')
    plt.title('Análise Comparativa de Desempenho', fontsize=18, weight='bold')
    plt.ylabel('Pontuação', fontsize=14)
    plt.xlabel('Modelo', fontsize=14)
    plt.ylim(0, 1.05)
    plt.xticks(rotation=0, ha='center', fontsize=12)
    plt.legend(title='Métricas', fontsize=11, loc='upper left')
    for p in ax.patches:
        ax.annotate(f'{p.get_height():.3f}', (p.get_x() + p.get_width()/2, p.get_height()),
                      ha='center', va='bottom', fontsize=10, color='black')
    plt.tight_layout()
    plt.savefig('comparacao_desempenho_final.png', dpi=300)
    plt.show()

# ========================================================
# 🔹 EXECUÇÃO PRINCIPAL
# ========================================================

def main():
    # --- Parâmetros ---
    NOME_DO_ARQUIVO = 'dataset_aumentado.csv'
    PAUSA_API = 0.5
    LIMIAR_TAU = 15
    AVERAGE_TYPE = 'weighted'

    # --- 1. Carga e Preparo dos Dados ---
    print(f"🔹 Carregando dados de: {NOME_DO_ARQUIVO}")
    try:
        df = pd.read_csv(NOME_DO_ARQUIVO)
    except FileNotFoundError:
        print(f"❌ ERRO: Arquivo '{NOME_DO_ARQUIVO}' não encontrado. Faça o upload para o ambiente do Colab.")
        return
    df['categoria'] = df['categoria'].astype(str)

    # --- 2. Pré-processamento de Texto ---
    print("🔹 Pré-processando textos com spaCy...")
    df['texto_limpo'] = df['texto'].apply(lambda x: preprocess_with_spacy(x, nlp))
    print("✅ Pré-processamento concluído.")

    # --- 3. Divisão de Dados e Vetorização ---
    X_original = df['texto']
    X_limpo = df['texto_limpo']
    y = df['categoria']
    indices = df.index

    # === CORREÇÃO APLICADA (Opção 1 - Rápida) ===
    # O parâmetro 'stratify=y' foi removido para evitar o ValueError e garantir a execução.
    X_train_limpo, X_test_limpo, y_train, y_test, indices_train, indices_test = train_test_split(
        X_limpo, y, indices, test_size=0.3, random_state=42
    )
    X_test_original = X_original.loc[indices_test]

    vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
    X_train_tfidf = vectorizer.fit_transform(X_train_limpo)
    X_test_tfidf = vectorizer.transform(X_test_limpo)

    # --- 4. Treinamento dos Modelos de Base ---
    print("🔹 Treinando modelos de base...")
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
    rf_model.fit(X_train_tfidf, y_train)

    svm_model = SVC(kernel='linear', probability=True, random_state=42, class_weight='balanced')
    svm_model.fit(X_train_tfidf, y_train)
    print("✅ Modelos de base treinados.")

    # --- 5. Avaliação dos Modelos ---
    resultados = []

    # Avaliação do RF
    y_pred_rf = rf_model.predict(X_test_tfidf)
    resultados.append({
        'Modelo': 'Random Forest', 'Acurácia': accuracy_score(y_test, y_pred_rf),
        'Precisão': precision_score(y_test, y_pred_rf, average=AVERAGE_TYPE, zero_division=0),
        'Recall': recall_score(y_test, y_pred_rf, average=AVERAGE_TYPE, zero_division=0),
        'F1-Score': f1_score(y_test, y_pred_rf, average=AVERAGE_TYPE, zero_division=0)
    })

    # Avaliação do SVM
    y_pred_svm = svm_model.predict(X_test_tfidf)
    resultados.append({
        'Modelo': 'SVM', 'Acurácia': accuracy_score(y_test, y_pred_svm),
        'Precisão': precision_score(y_test, y_pred_svm, average=AVERAGE_TYPE, zero_division=0),
        'Recall': recall_score(y_test, y_pred_svm, average=AVERAGE_TYPE, zero_division=0),
        'F1-Score': f1_score(y_test, y_pred_svm, average=AVERAGE_TYPE, zero_division=0)
    })

    # Avaliação do CHyPS
    print("🔹 Avaliando o CHyPS...")
    y_pred_chyps = []
    for texto_original, texto_limpo in zip(X_test_original, X_test_limpo):
        pred = chyps_classify(texto_original, texto_limpo, rf_model, vectorizer, LIMIAR_TAU)
        y_pred_chyps.append(pred)
        time.sleep(PAUSA_API)

    resultados.append({
        'Modelo': 'CHyPS', 'Acurácia': accuracy_score(y_test, y_pred_chyps),
        'Precisão': precision_score(y_test, y_pred_chyps, average=AVERAGE_TYPE, zero_division=0),
        'Recall': recall_score(y_test, y_pred_chyps, average=AVERAGE_TYPE, zero_division=0),
        'F1-Score': f1_score(y_test, y_pred_chyps, average=AVERAGE_TYPE, zero_division=0)
    })

    # --- 6. Apresentação dos Resultados ---
    resultados_df = pd.DataFrame(resultados).set_index('Modelo')
    print("\n" + "="*40)
    print("📊 RESULTADOS FINAIS 📊")
    print("="*40)
    print(resultados_df)

    plotar_resultados(resultados_df)

    print("🔹 Salvando resultados em CSV...")
    resultados_df.to_csv('resultados_chyps.csv')

    print("🔹 Gerando Matriz de Confusão para o CHyPS...")
    all_classes = sorted(y.unique())
    ConfusionMatrixDisplay.from_predictions(y_test, y_pred_chyps, labels=all_classes, cmap='Blues')
    plt.title('Matriz de Confusão - CHyPS')
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.savefig('matriz_confusao_chyps.png', dpi=300)
    plt.show()

    print("\n✅ Fim da execução 🚀")

# Ponto de entrada do script
if __name__ == '__main__':
    main()

🔹 Ambiente: Google Colab detectado.
⚠️ Modelo 'pt_core_news_lg' não encontrado. Tentando 'pt_core_news_md'...
✅ Modelo spaCy 'pt_core_news_md' carregado.
⚠️ Variável de ambiente 'GROQ_API_KEY' não encontrada.
✅ Chave da API da Groq carregada do Colab Secrets.
✅ Cliente da API Groq configurado.
🔹 Carregando dados de: dataset_aumentado.csv
🔹 Pré-processando textos com spaCy...
✅ Pré-processamento concluído.
🔹 Treinando modelos de base...
✅ Modelos de base treinados.
🔹 Avaliando o CHyPS...
